In [36]:
import os
import pandas as pd
import numpy as np
import copy
# import hyperloglog
from datasketch import HyperLogLog

# Points
For now i am assuming that primary key is only a single attribute

In [ ]:
class Attribute:
    def __init__(self, table_name, attribute_name, values):
        self.table_name = table_name
        self.attribute_name = attribute_name
        self.values = values

        self.uniquness = self.estUniqueness()
        self.cardinality=1
        self.value_length = 1/max(1, max([len(x) for x in values]) - 8)
        self.position = 0
        self.suffix = self.check_suffix()
        
        self.dependent = [] # It's dependent attributes
        self.reference = [] # Attributes that it references from 

    def estUniqueness(self):
        hll = HyperLogLog()
        total = 0
        
        for value in self.values:
            hll.update(str(value).encode('utf8'))
            total +=1
        # print(f"{hll.count()=}")
        
        return hll.count() / total
    
    def check_suffix(self, suffix_list=["key", 'id', 'nr', 'no']):
        for suffix in suffix_list:
            if suffix in self.attribute_name:
                return 1

In [38]:
def load_csv_files(directory_path):
    attributes = {}

    csv_files = [f for f in os.listdir(directory_path)]

    print(f"Found {len(csv_files)} \n CSV files: {csv_files}")    

    for filename in csv_files:
        file_path = os.path.join(directory_path, filename)
        table_name = os.path.splitext(filename)[0]

        df = pd.read_csv(file_path)
        df = df
        print(f"Processing {filename}: {df.shape[0]} rows, {df.shape[1]} columns")

        for i, column in enumerate(df.columns):
            non_null_values = df[column].dropna().tolist()
            if non_null_values:
                attr = Attribute(table_name, column, non_null_values)
                attr.position = 1/(i+1)
                attributes[f"{table_name}.{column}"] = attr
                print(f"Added attribute: {attr.table_name}.{attr.attribute_name} Total Values: {len(attr.values)}")
                print(attr.position)

    return attributes
    
attributes = load_csv_files("/home/haseeb/Desktop/EKAI/ERD_automation/Dataset/train/menagerie-db")            
            

Found 2 
 CSV files: ['event.csv', 'pet.csv']
Processing event.csv: 10 rows, 4 columns
Added attribute: event.name Total Values: 10
1.0
Added attribute: event.date Total Values: 10
0.5
Added attribute: event.type Total Values: 10
0.3333333333333333
Added attribute: event.remark  Total Values: 8
0.25
Processing pet.csv: 8 rows, 6 columns
Added attribute: pet.lexicon Total Values: 8
1.0
Added attribute: pet.owner Total Values: 8
0.5
Added attribute: pet.species Total Values: 8
0.3333333333333333
Added attribute: pet.sex Total Values: 8
0.25
Added attribute: pet.birth Total Values: 8
0.2
Added attribute: pet.death Total Values: 8
0.16666666666666666


In [ ]:
def read_IND(file_path):
    '''
        fills up the dependent and reference array of attributes
    '''
    with open(file_path, "r") as f:
        for line in f:
            vars = line.strip().split("=")
            attributes[vars[0]].references = attributes[vars[1]]
            attributes[vars[1]].dependent = attributes[vars[0]]
read_IND("/home/haseeb/Desktop/EKAI/ERD_automation/codes/inclusionDependencyWithSpider/spider_results/menagerie-db.txt")

event.name=pet.lexicon



In [39]:
attributes["event.name"].uniquness

np.float64(0.8127666768532549)